# EE 599 HW 4: GPU CUDA Programming

Your task in this Colab notebook is to fill out the sections that are specified by **TODO** (please search the keyword `TODO` to make sure you do not miss any).

Prerequisites: set the runtime type to GPU. (Runtime -> Change Runtime Type)

The `nvidia-smi` cli tells you about the GPU information on Colab.

In [ ]:
!nvidia-smi

Thu Mar 21 01:27:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

The GPU compiler for c++ from Nvidia is called `nvcc`, and is already installed on Colab.

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


## Vector Add

Similar to the previous assignment, we use `%%writefile` command to save the content of a notebook cell directly into a file, which can then be compiled and executed using command-line instructions.

The example file `vector_add.cu` implements the vector addition using 64K threads with CUDA programming.

In [ ]:
%%writefile vector_add.cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>

#define size 65536

__global__ void vector_add(int *A, int *B, int *C) {
    int my_id = blockIdx.x * blockDim.x + threadIdx.x;
    C[my_id] = A[my_id] + B[my_id];
}

int main() {
    int i;
    int *A = (int *)malloc(sizeof(int) * size);
    int *B = (int *)malloc(sizeof(int) * size);
    int *C = (int *)malloc(sizeof(int) * size);

    for (i = 0; i < size; i++) {
        A[i] = 1;
        B[i] = 2;
    }

    int *gpu_A, *gpu_B, *gpu_C;
    cudaMalloc((void **)&gpu_A, sizeof(int) * size);
    cudaMalloc((void **)&gpu_B, sizeof(int) * size);
    cudaMalloc((void **)&gpu_C, sizeof(int) * size);

    struct timespec start, stop;
    double time;

    cudaMemcpy(gpu_A, A, sizeof(int) * size, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_B, B, sizeof(int) * size, cudaMemcpyHostToDevice);

    dim3 dimGrid(64);
    dim3 dimBlock(1024);

    if (clock_gettime(CLOCK_REALTIME, &start) == -1) {
        perror("clock gettime");
    }
    vector_add<<<dimGrid, dimBlock>>>(gpu_A, gpu_B, gpu_C);
    cudaMemcpy(C, gpu_C, sizeof(int) * size, cudaMemcpyDeviceToHost);

    if (clock_gettime(CLOCK_REALTIME, &stop) == -1) {
        perror("clock gettime");
    }
    time = (stop.tv_sec - start.tv_sec) + (double)(stop.tv_nsec - start.tv_nsec) / 1e9;

    printf("Execution time = %f sec\n", time);

    for (i = 0; i < 10; i++) {
        printf("C[%d]=%d ", i, C[i]);
    }

    free(A);
    free(B);
    free(C);
    cudaFree(gpu_A);
    cudaFree(gpu_B);
    cudaFree(gpu_C);
    return 0;
}

Overwriting vector_add.cu


Compile and execute the code.

In [ ]:
!nvcc vector_add.cu -o vector_add.out && ./vector_add.out

Execution time = 0.118620 sec
C[0]=3 C[1]=3 C[2]=3 C[3]=3 C[4]=3 C[5]=3 C[6]=3 C[7]=3 C[8]=3 C[9]=3 

## Matrix Multiplacation

### **TODO 1:**

Implement unoptimized matrix multiplication using global memory only:

- Thread block configuration: 16 × 16
- Grid configuration: 64 × 64

In [ ]:
%%writefile mat_mul.cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>

#define n 1024

// TODO: Write GPU kernel to perform matrix multiplication
__global__ void matrixMultiplication(int *A, int *B, int *C){
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int sum = 0;

    if(col < n && row < n){
        for(int i = 0; i < n;i++){
            sum += A[row * n + i] * B[i * n + col];
        }
        C[row * n + col] = sum;
    }
}

int main() {
    int i, j;
    int *A = (int *) malloc(sizeof(int) * n * n);
    int *B = (int *) malloc(sizeof(int) * n * n);
    int *C = (int *) malloc(sizeof(int) * n * n);

    for (i = 0; i < n; i++) {
        for (j = 0; j < n; j++) {
            A[i*n + j] = i;
            B[i*n + j] = i + j;
            C[i*n + j] = 0;
        }
    }

    // TODO: Allocate device memory, use variable names: gpu_A, gpu_B and gpu_C
    int *gpu_A, *gpu_B, *gpu_C;
    cudaMalloc((void **)&gpu_A, sizeof(int) * n * n);
    cudaMalloc((void **)&gpu_B, sizeof(int) * n * n);
    cudaMalloc((void **)&gpu_C, sizeof(int) * n * n);

    // TODO: Transfer data to device
    cudaMemcpy(gpu_A, A, sizeof(int) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_B, B, sizeof(int) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_C, C, sizeof(int) * n * n, cudaMemcpyHostToDevice);

    // TODO: Set grid and block sizes
    dim3 dimGrid(64, 64);
    dim3 dimBlock(16, 16);

    struct timespec start, stop;
    double time;

    if( clock_gettime( CLOCK_REALTIME, &start) == -1 ) { perror( "clock gettime" );}

    // TODO: Launch kernel
    matrixMultiplication<<<dimGrid, dimBlock>>>(gpu_A, gpu_B, gpu_C);

    // TODO: Transfer data back to host
    cudaMemcpy(C, gpu_C, sizeof(int) * n * n, cudaMemcpyDeviceToHost);

    if( clock_gettime( CLOCK_REALTIME, &stop) == -1 ) { perror( "clock gettime" );}
    time = (stop.tv_sec - start.tv_sec)+ (double)(stop.tv_nsec - start.tv_nsec)/1e9;

    // Print results
    printf("Number of FLOPs = %llu, Execution time = %f sec,\n%lf MFLOPs per sec\n",
        2ULL * n * n * n, time, 1 / time / 1e6 * 2 * n * n * n);

    printf("C[100][100]=%d\n", C[100*n + 100]);

    free(A);
    free(B);
    free(C);
    cudaFree(gpu_A);
    cudaFree(gpu_B);
    cudaFree(gpu_C);
    return 0;
}

Overwriting mat_mul.cu


In [ ]:
!nvcc mat_mul.cu -o mat_mul.out && ./mat_mul.out

Number of FLOPs = 2147483648, Execution time = 0.126278 sec,
17006.035126 MFLOPs per sec
C[100][100]=62617600


### **TODO 2**:

Implement block matrix multiplication using shared memory.
- Thread block configuration: 32 × 32
- Grid configuration: 32 × 32

In [7]:
%%writefile blocked_mat_mul.cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>

#define n 1024
#define block_size 32

// TODO: Write GPU kernel to perform matrix multiplication
__global__ void matrixMulShared(int *A, int *B, int *C, int width) {
    __shared__ int As[block_size][block_size];
    __shared__ int Bs[block_size][block_size];

    int bx = blockIdx.x, by = blockIdx.y,
        tx = threadIdx.x, ty = threadIdx.y,
        Row = by * block_size + ty,
        Col = bx * block_size + tx;
    int sum = 0;

    for (int m = 0; m < width/block_size; ++m) {
        As[ty][tx] = A[Row*width + (m*block_size + tx)];
        Bs[ty][tx] = B[(m*block_size + ty)*width + Col];
        __syncthreads();

        for (int k = 0; k < block_size; ++k) {
            sum += As[ty][k] * Bs[k][tx];
        }
        __syncthreads();
    }
    C[Row*width + Col] = sum;
}

int main() {
    int i, j;
    int *A = (int *) malloc(sizeof(int)*n*n);
    int *B = (int *) malloc(sizeof(int)*n*n);
    int *C = (int *) malloc(sizeof(int)*n*n);

    for (i = 0; i < n; i++) {
        for (j = 0; j < n; j++) {
            A[i*n + j] = i;
            B[i*n + j] = i + j;
            C[i*n + j] = 0;
        }
    }

    // TODO: Allocate device memory, use variable names: gpu_A, gpu_B and gpu_C
    int *gpu_A, *gpu_B, *gpu_C;
    cudaMalloc((void **)&gpu_A, sizeof(int)*n*n);
    cudaMalloc((void **)&gpu_B, sizeof(int)*n*n);
    cudaMalloc((void **)&gpu_C, sizeof(int)*n*n);

    // TODO: Transfer data to device
    cudaMemcpy(gpu_A, A, sizeof(int)*n*n, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_B, B, sizeof(int)*n*n, cudaMemcpyHostToDevice);

    // TODO: Set grid and block sizes
    dim3 dimBlock(block_size, block_size);
    dim3 dimGrid(n / block_size, n / block_size);

    struct timespec start, stop;
    double time;

    if( clock_gettime( CLOCK_REALTIME, &start) == -1 ) { perror( "clock gettime" );}

    // TODO: Launch kernel
    matrixMulShared<<<dimGrid, dimBlock>>>(gpu_A, gpu_B, gpu_C, n);

    // TODO: Transfer data back to host
    cudaMemcpy(C, gpu_C, sizeof(int)*n*n, cudaMemcpyDeviceToHost);

    if( clock_gettime( CLOCK_REALTIME, &stop) == -1 ) { perror( "clock gettime" );}
    time = (stop.tv_sec - start.tv_sec)+ (double)(stop.tv_nsec - start.tv_nsec)/1e9;

    // Print results
    printf("Number of FLOPs = %llu, Execution time = %f sec,\n%lf MFLOPs per sec\n",
        2ULL * n * n * n, time, 1 / time / 1e6 * 2 * n * n * n);

    printf("C[100][100]=%d\n", C[100*n + 100]);

    free(A);
    free(B);
    free(C);
    cudaFree(gpu_A);
    cudaFree(gpu_B);
    cudaFree(gpu_C);
    return 0;
}

Writing blocked_mat_mul.cu


In [8]:
!nvcc blocked_mat_mul.cu -o blocked_mat_mul.out && ./blocked_mat_mul.out

Number of FLOPs = 2147483648, Execution time = 0.052455 sec,
40939.721763 MFLOPs per sec
C[100][100]=62617600
